# Bank Heist Project

### 1. Initialization and Constants

- Maximum time allowed for the heist is 24 hours, meanwhile we travel between banks at a speed of 30km / h. 
- The coordinates are in kilometers so (1, 1) and (1,2) are one kilometer apart. 
- The solution has to run on a common laptop under 3 minutes for the 10,000 cities dataset
- We must finish or start at the escape zones which is coordinate (0,0) 

In [98]:
import pandas as pd
import numpy as np
import time
from scipy.spatial.distance import cdist

df = pd.read_csv ('bank_data.csv')
print(df)

        id  x_coordinate  y_coordinate  money  time (hr)
0        0      2.851925      1.201712  29700   0.273403
1        1      4.150372     -2.654334   6500   0.064040
2        2     -1.494092     -1.230419  89400   0.127458
3        3      1.271326     -0.088520  96100   1.315029
4        4      2.471113     -0.592810  41100   0.164393
...    ...           ...           ...    ...        ...
9995  9995     -2.472127      4.787304   2100   0.494904
9996  9996      1.034968     -4.321240  54900   0.492866
9997  9997     -0.512720     -2.949173  32200   0.933033
9998  9998      2.626841     -3.482923   5100   0.551312
9999  9999     -3.992507      0.090112  21000   1.107299

[10000 rows x 5 columns]


### 2. Distance Calculation

### 3. Greedy Algorithm

In [108]:
def greedy_algo(df):
    start_time = time.time()
    n = len(df)
    current_time = 0 
    current_position = (0,0)
    visited = []
    total_money = 0
    money_by_bank = []
    
    while current_time <24 * 60 and time.time() - start_time < 180:
        min_distance = np.inf
        next_bank = None
        
        for idx, row in df.iterrows():
            if idx in visited:
                continue
#Verifying the distance and the travel time in minutes                
            distance = np.linalg.norm(np.array(current_position) - np.array((row['x_coordinate'], row['y_coordinate'])))
            travel_time = distance / 30 * 60
#Verifying if the visit is in the time limit                               
            if current_time+travel_time+row['time (hr)']*60 <24 * 60:
                if distance < min_distance:
                    min_distance = distance
                    next_bank = idx
                    
        if next_bank is None:
            break
#Compiling the visited banks and the time left           
        visited.append(next_bank)
        money_earned = df.loc[next_bank, 'money']
        total_money += money_earned
        money_by_bank.append((next_bank, money_earned)) 
        current_time += (min_distance / 30 * 60) + (df.loc[next_bank, 'time (hr)'] * 60)
        current_position = (df.loc[next_bank, 'x_coordinate'], df.loc[next_bank, 'y_coordinate'])
    
    return visited, total_money, money_by_bank


                                 
result, total_money, money_by_bank = greedy_algo(df)
print("Visited Banks:", result)
print("Total Money Made:", total_money)
print("Bank ID & Money Earned:", money_by_bank)

Visited Banks: [4765, 8789, 6131, 7271, 8021, 9689, 3859, 6366, 6000, 9440, 6824, 6548, 9941, 4469, 3568, 5517, 3746, 640, 6787, 5303, 959, 7676, 8347, 2450, 6636, 3966, 1455, 1150, 4422, 9066, 6254, 3914, 8562, 6401]
Total Money Made: 853200
Bank ID & Money Earned: [(4765, 12900), (8789, 93200), (6131, 4300), (7271, 8200), (8021, 13500), (9689, 9400), (3859, 3100), (6366, 600), (6000, 29200), (9440, 500), (6824, 9800), (6548, 6200), (9941, 3600), (4469, 11300), (3568, 95800), (5517, 200), (3746, 40500), (640, 3300), (6787, 19500), (5303, 40400), (959, 87000), (7676, 400), (8347, 8400), (2450, 1100), (6636, 2400), (3966, 600), (1455, 78800), (1150, 30900), (4422, 77700), (9066, 300), (6254, 55100), (3914, 25200), (8562, 79100), (6401, 700)]


### 4. Implementation Details

In [110]:
def robber_algo(df):
    start_time = time.time()
    n = len(df)
    best_route = []
    max_banks_visited = 0
    total_money = 0
    money_by_bank = []

    for start_bank in df.index:
        current_time = 0
        current_position = (0, 0)
        visited = []
        
        while current_time < 24 * 60 and time.time() - start_time < 180:
            min_distance = np.inf
            next_bank = None
            
            for idx, row in df.iterrows():
                if idx in visited:
                    continue
                
                distance = np.linalg.norm(np.array(current_position) - np.array((row['x_coordinate'], row['y_coordinate'])))
                travel_time = distance / 30 * 60  # Convert hours to minutes
                
                if current_time + travel_time + row['time (hr)'] * 60 < 24 * 60:
                    if distance < min_distance:
                        min_distance = distance
                        next_bank = idx
            
            if next_bank is None:
                break
            
            visited.append(next_bank)
            money_earned = df.loc[next_bank, 'money']
            total_money += money_earned
            money_by_bank.append((next_bank, money_earned)) 
            current_time += (min_distance / 30 * 60) + (df.loc[next_bank, 'time (hr)'] * 60)
            current_position = (df.loc[next_bank, 'x_coordinate'], df.loc[next_bank, 'y_coordinate'])
        
        if len(visited) > max_banks_visited:
            max_banks_visited = len(visited)
            best_route = visited
    
    return best_route, total_money,money_by_bank


                                 
result, total_money,money_by_bank = robber_algo(df)
print("Visited Banks:", result)
print("Total Money Made:", total_money)
print("Bank ID & Money Earned:", money_by_bank)

Visited Banks: [4765, 8789, 6131, 7271, 8021, 9689, 3859, 6366, 6000, 9440, 6824, 6548, 9941, 4469, 3568, 5517, 3746, 640, 6787, 5303, 959, 7676, 8347, 2450, 6636, 3966, 1455, 1150, 4422, 9066, 6254, 3914, 8562, 6401]
Total Money Made: 4959100
Bank ID & Money Earned: [(4765, 12900), (8789, 93200), (6131, 4300), (7271, 8200), (8021, 13500), (9689, 9400), (3859, 3100), (6366, 600), (6000, 29200), (9440, 500), (6824, 9800), (6548, 6200), (9941, 3600), (4469, 11300), (3568, 95800), (5517, 200), (3746, 40500), (640, 3300), (6787, 19500), (5303, 40400), (959, 87000), (7676, 400), (8347, 8400), (2450, 1100), (6636, 2400), (3966, 600), (1455, 78800), (1150, 30900), (4422, 77700), (9066, 300), (6254, 55100), (3914, 25200), (8562, 79100), (6401, 700), (4765, 12900), (8789, 93200), (6131, 4300), (7271, 8200), (8021, 13500), (9689, 9400), (3859, 3100), (6366, 600), (6000, 29200), (9440, 500), (6824, 9800), (6548, 6200), (9941, 3600), (4469, 11300), (3568, 95800), (5517, 200), (3746, 40500), (640, 

### 5. Output

In [120]:
import pandas as pd
import numpy as np

def robber_algorithm(df):
    # Sort banks by money/time
    df['profit'] = df['money'] / df['time (hr)']
    df.sort_values(by='profit', ascending=False, inplace=True)

    visited_banks = []
    time = 24 * 60  # Convert hours to minutes
    current_position = (0, 0)
    total_money = 0

    # Greedy approach
    for _, row in df.iterrows():
        distance = np.sqrt((row['x_coordinate'] - current_position[0])**2 +
                           (row['y_coordinate'] - current_position[1])**2)
        travel_time = distance / 30 * 60 # Travel time in minutes
        robbing_time = row['time (hr)'] * 60
        
        
        if time >= travel_time + robbing_time:
            visited_banks.append(row['id'])
            total_money += row['money']
            time -= travel_time + robbing_time
            current_position = (row['x_coordinate'], row['y_coordinate'])
        else :
                break

    # Add escape zone
    distance_to_escape = np.sqrt(current_position[0]**2 + current_position[1]**2)
    travel_time_to_escape = distance_to_escape / 30 * 60  # Travel time in minutes

    if time >= travel_time_to_escape:
        visited_banks.append(0)
        time -= travel_time_to_escape

    return visited_banks, total_money

result, total_money = robber_algorithm(df)
print("Visited Banks:", result)
print("Total Money Earned:", total_money)


Visited Banks: [3613.0, 9546.0, 3803.0, 6528.0, 9583.0, 5933.0, 9195.0, 4757.0, 8550.0, 4499.0, 9241.0, 7343.0, 3798.0, 7544.0, 4762.0, 8436.0, 8562.0, 6097.0, 487.0, 2928.0, 5135.0, 865.0, 3914.0, 4987.0, 3466.0, 7560.0, 8579.0, 8690.0, 8469.0, 433.0, 524.0, 6468.0, 1914.0, 1757.0, 8287.0, 7258.0, 4725.0, 8206.0, 4605.0, 7064.0, 3926.0, 9401.0, 9736.0, 2331.0, 9653.0, 7074.0, 2346.0, 8966.0, 4696.0, 8022.0, 4789.0, 6740.0, 9881.0, 70.0, 2656.0, 951.0, 209.0, 58.0, 9228.0, 3193.0, 8525.0, 5610.0, 2037.0, 4287.0, 6022.0, 2827.0, 8849.0, 9378.0, 6104.0, 2741.0, 7701.0, 8355.0, 5563.0, 3005.0, 7665.0, 6987.0, 5719.0, 2729.0, 1997.0, 8286.0, 4906.0, 613.0, 790.0, 7087.0, 5126.0, 5562.0, 3516.0, 2243.0, 6759.0, 3026.0, 1372.0, 2442.0, 1733.0, 8375.0, 7764.0, 7649.0, 7689.0, 9880.0, 3297.0, 4794.0, 3089.0, 7583.0, 5725.0, 517.0, 4345.0, 2521.0, 2190.0, 5155.0, 9049.0, 444.0, 6712.0, 8908.0, 8703.0, 7877.0, 7595.0, 8231.0, 9275.0, 4465.0, 5295.0, 5296.0, 299.0, 8846.0, 6254.0, 507.0, 6535.0, 